In [1]:
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import numpy as np

#from ray.data import ActorPoolStrategy

class EmbedChunks:
    def __init__(self):
       
        self.embedding_model = OpenAIEmbeddings(
            model="text-embedding-ada-002",
            openai_api_base=os.getenv("OPENAI_API_BASE"),
            openai_api_key=os.getenv("OPENAI_API_KEY"))
      

    def process_batch(self, batch):
        embeddings = self.embedding_model.embed_documents(batch["question"])
       
        return pd.DataFrame({"question": batch["question"], "solution": batch["solution"], "embeddings": embeddings})

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
from datasets import load_dataset

openai_api_key = os.getenv("OPENAI_API_KEY")
from langchain.embeddings.openai import OpenAIEmbeddings
import pandas as pd
data_path = "../dataset/data.jsonl"

df = pd.read_json(data_path, lines=True)
# embed_model = OpenAIEmbeddings(model="text-embedding-ada-002",openai_api_key=openai_api_key)
# texts = df['question']


 

# dataset = load_dataset(
#     data_path,
#     split="test"
# )
# Instantiate the EmbedChunks class
embedder = EmbedChunks()

# Apply the __call__ method to each example in the dataset
# embeded_chunk = dataset.map(embedder, batched=True)



# Specify the batch size
batch_size = 100

# Split the DataFrame into batches
batches = [df.iloc[i:i+batch_size] for i in range(0, len(df), batch_size)]

# Apply the function to each batch
processed_batches = [embedder.process_batch(batch) for batch in batches]

# Concatenate the processed batches back into a single DataFrame
embeded_chunk = pd.concat(processed_batches, ignore_index=True)






# # Embed chunks
# embeded_chunk = df.map_batches( 
#     EmbedChunks,
#     batch_size=100, 
#     # num_gpus=1
#     )
# embeded_chunk = embed_model.embed_documents(texts)
# len(embeded_chunk), len(embeded_chunk[0])
#for i, embeded_chunk in enumerate(embeded_chunk):
    #print(f"Embedded representation for chunk {i+1}:")
print(embeded_chunk.iloc[0])



C:\Users\HP\.conda\envs\week6_env\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


question      An image is represented by an m x n integer gr...
solution      class Solution {\npublic:\n    void bfs(int pr...
embeddings    [0.019774852960477488, 0.013059384313380287, -...
Name: 0, dtype: object


In [ ]:
from tqdm import tqdm
import psycopg2

class StoreResults:
    def __call__(self, batch):
        # Establish a connection to the database using the DB_CONNECTION_STRING from the environment variables
        with psycopg.connect(os.environ["DB_CONNECTION_STRING"]) as conn:
            # Register the vector type in the database (assuming it's a custom vector type)
            register_vector(conn)
            with conn.cursor() as cur:
                # Iterate over each element in the batch and insert the data into the database
                for question, solution, embeded_chunk in zip(batch["question"], batch["solution"], batch["embeded_chunk"]):
                    cur.execute("INSERT INTO document (question, solution, embeded_chunk) VALUES (%s, %s, %s)",
                                (question, solution, embeded_chunk,))
        # Return an empty dictionary
        return {}

store_results = StoreResults()  # Instantiate the StoreResults object

# Iterate over the elements of embeded_chunk using tqdm
for _, row in tqdm(embeded_chunk.iterrows(), total=len(embeded_chunk), desc="Processing"): 
    StoreResults()(row)

In [ ]:
# Index data
embedded_chunks.map_batches(
    StoreResults,
    batch_size=128,
    num_cpus=1,
    compute=ActorPoolStrategy(size=28),
).count()